In [4]:
import pandas as pd

In [5]:
import pg8000

In [7]:
conn = pg8000.connect(user='dot_student', host='training.c1erymiua9dx.us-east-1.rds.amazonaws.com', port=5432, database='training', password='qgis')

In [8]:
df = pd.read_sql("select * from noise_311", conn)


In [31]:
df.describe()

/usr/local/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,b'unique_key',b'x_coordinate',b'y_coordinate',b'latitude',b'longitude',open_time
count,3.761500e+04,3.736300e+04,37363.000000,37363.000000,37363.000000,37615
mean,2.848004e+07,1.002332e+06,212926.874207,40.751076,-73.934715,0 days 07:44:22.532500
std,1.892448e+05,1.727615e+04,29488.008628,0.080940,0.062316,1 days 12:06:09.012799
min,2.815695e+07,9.143810e+05,121683.000000,40.500415,-74.251277,0 days 00:01:00
25%,2.831167e+07,NaN,NaN,NaN,NaN,0 days 00:54:00
50%,2.849327e+07,NaN,NaN,NaN,NaN,0 days 02:02:00
75%,2.865054e+07,NaN,NaN,NaN,NaN,0 days 04:15:00
max,2.879956e+07,1.066985e+06,271639.000000,40.912218,-73.701409,48 days 05:45:00


In [14]:
df.head()

,b'unique_key',b'created_date',b'closed_date',b'agency',b'agency_name',b'complaint_type',b'descriptor',b'location_type',b'incident_zip',b'incident_address',...,b'bridge_highway_name',b'bridge_highway_direction',b'road_ramp',b'bridge_highway_segment',b'garage_lot_name',b'ferry_direction',b'ferry_terminal_name',b'latitude',b'longitude',b'location'
0,28792167,2014-08-31 23:59:00,2014-09-01 03:52:00,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11222,200 KINGSLAND AVENUE,...,None,None,None,None,None,None,None,40.723888,-73.941349,"(40.723888303549415, -73.94134888943505)"
1,28789088,2014-08-31 23:56:00,2014-09-01 06:17:00,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,11234,FLATLANDS AVENUE,...,None,None,None,None,None,None,None,40.619489,-73.938051,"(40.61948901090983, -73.93805104516916)"
2,28791854,2014-08-31 23:54:00,2014-09-01 01:29:00,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,10002,161 LUDLOW STREET,...,None,None,None,None,None,None,None,40.721410,-73.987694,"(40.72141034382407, -73.98769444021134)"
3,28789936,2014-08-31 23:52:00,2014-09-01 02:53:00,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10033,624 WEST 182 STREET,...,None,None,None,None,None,None,None,40.850167,-73.933972,"(40.85016671877659, -73.93397220795968)"
4,28789931,2014-08-31 23:47:00,2014-09-01 01:06:00,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11217,525 DEAN STREET,...,None,None,None,None,None,None,None,40.681208,-73.972775,"(40.68120794066068, -73.97277535440028)"


In [19]:
type(df[b'created_date'][0])

pandas.tslib.Timestamp

In [20]:
df['open_time'] = df[b'closed_date'] - df[b'created_date']

In [23]:
df['open_time'].describe()

count                     37615
mean     0 days 07:44:22.532500
std      1 days 12:06:09.012799
min             0 days 00:01:00
25%             0 days 00:54:00
50%             0 days 02:02:00
75%             0 days 04:15:00
max            48 days 05:45:00
Name: open_time, dtype: object

In [24]:
iqr = df['open_time'].quantile(q=0.75)- df['open_time'].quantile(q=0.25)
iqr

Timedelta('0 days 03:21:00')

In [25]:
ual = df['open_time'].quantile(q=0.75) + (iqr * 1.5)
ual

Timedelta('0 days 09:16:30')

In [27]:
lal = df['open_time'].quantile(q=0.25) - (iqr*1.5) 
lal

Timedelta('-1 days +19:52:30')

In [28]:
outlier = df[df['open_time']>ual]

In [33]:
outlier

,b'unique_key',b'created_date',b'closed_date',b'agency',b'agency_name',b'complaint_type',b'descriptor',b'location_type',b'incident_zip',b'incident_address',...,b'bridge_highway_direction',b'road_ramp',b'bridge_highway_segment',b'garage_lot_name',b'ferry_direction',b'ferry_terminal_name',b'latitude',b'longitude',b'location',open_time
17,28790593,2014-08-31 23:27:00,2014-09-01 12:59:00,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,10469,3480 CORSA AVENUE,...,None,None,None,None,None,None,40.877892,-73.847907,"(40.87789192345351, -73.84790671078991)",0 days 13:32:00
41,28790592,2014-08-31 22:54:00,2014-09-01 09:14:00,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11377,50-20 31 AVENUE,...,None,None,None,None,None,None,40.757207,-73.908314,"(40.75720724921344, -73.90831381995812)",0 days 10:20:00
74,28792643,2014-08-31 22:22:00,2014-09-01 16:08:00,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,11211,286 GRAHAM AVENUE,...,None,None,None,None,None,None,40.712098,-73.943974,"(40.712098120304915, -73.94397441662797)",0 days 17:46:00
77,28789677,2014-08-31 22:16:00,2014-09-02 12:14:00,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",None,10038,100 BEEKMAN STREET,...,None,None,None,None,None,None,40.708275,-74.003556,"(40.708274520328345, -74.00355640309049)",1 days 13:58:00
105,28788623,2014-08-31 21:42:00,2014-09-02 19:40:00,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),None,10314,MCVEIGH AVENUE,...,None,None,None,None,None,None,40.587838,-74.151515,"(40.58783790256183, -74.15151548694752)",1 days 21:58:00
113,28792644,2014-08-31 21:28:00,2014-09-01 07:19:00,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,10009,None,...,None,None,None,None,None,None,40.722672,-73.982965,"(40.722672330009374, -73.98296454007392)",0 days 09:51:00
120,28790718,2014-08-31 21:20:00,2014-09-02 12:10:00,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",None,11223,1663 WEST FIRST STREET,...,None,None,None,None,None,None,40.607327,-73.974739,"(40.60732702196091, -73.97473945903694)",1 days 14:50:00
129,28788621,2014-08-31 21:10:00,2014-09-02 12:09:00,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",None,11385,88-20 AUBREY AVENUE,...,None,None,None,None,None,None,40.710420,-73.863770,"(40.71041996957435, -73.86377017318794)",1 days 14:59:00
158,28790118,2014-08-31 20:21:00,2014-09-01 05:49:00,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10026,280 WEST 117 STREET,...,None,None,None,None,None,None,40.804826,-73.954249,"(40.8048256171524, -73.95424873879473)",0 days 09:28:00
161,28791797,2014-08-31 15:33:00,2014-09-02 11:51:00,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",None,11385,2121 GROVE STREET,...,None,None,None,None,None,None,40.708974,-73.904932,"(40.70897365975388, -73.9049319292409)",1 days 20:18:00
